In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import iob_ia.utils.segment_extra as segex
from iob_ia.utils import classify, io_utils, segment, visualise

## Define path to iamge

In [ ]:
path_base = 'G:/20250211_MarcDu/'
path_base = path_base + '20x_Testfile_405(DAPI)_488(CD45)_546(CD3)_647(CD31)/'
path_tif = path_base + 'DAPI_ch3-20x.tif'
path_nd2 = path_base + '3-20x.nd2'

In [ ]:
# Specify the channels (0-based indexing)
dapi_ch = 0
cd45_ch = 1
# channels of interest
oste_ch = 2
cd31_ch = 3

In [ ]:
img_nd2 = io_utils.read_image(path_nd2)

In [ ]:
print(img_nd2.get_data().shape)
print(img_nd2.physical_pixel_sizes)

## Segment using cellpose

In [ ]:
if os.path.exists(io_utils.gen_out_path(path_tif, 'cp_seg')):
    mask_cp = io_utils.read_labels(io_utils.gen_out_path(path_tif, 'cp_seg'))
else:
    mask_cp = segment.segment_3d_cellpose(
        img_nd2.get_data()[dapi_ch],
        #model_path = 'cyto3', # Default
        #anisotropy=img_nd2.physical_pixel_sizes,
        #flow3D_smooth = 0, # Default
        #cellprob_threshold = 0.0, # Default
    )
print(mask_cp.shape)

In [ ]:
# save labels
save_path = io_utils.gen_out_path(path_nd2, 'cp_seg')
io_utils.save_labels(mask_cp, save_path)

## Filter lable image on features
shape features: 'area, 'euler_number' and 'extent'
intensity features: "intensity_max", "intensity_mean", "intensity_min"

In [ ]:
# Measure features of all labels created with cellpose, using calibrated units
property_table = segment.measure_props(
    img_label=mask_cp, img_intensity=img_nd2.get_data(),
    voxel_size=img_nd2.physical_pixel_sizes
)
property_table = segment.measure_props(
    img_label=mask_cp, voxel_size=img_nd2.physical_pixel_sizes
)

In [ ]:
label2remove = segment.filter_shape(
    mask_cp, 'projected_circularity', min_val=0.55, props_table=property_table
)
label2remove = segment.filter_shape(
    mask_cp, 'area', min_val=900, labels_to_remove=label2remove,
    props_table=property_table
) # (10x10x9)um^3

In [ ]:
nuclei = segment.remove_label_objects(
    mask_cp, labels=label2remove, relabel=False
)

In [ ]:
save_path = io_utils.gen_out_path(path_tif, 'filtered')
io_utils.save_labels(nuclei, save_path)

## Visualise

In [ ]:
# add multi channel nd2
visualise.add_multichannel_image(
    img=img_nd2.get_data(), name='image',
    channel_names=img_nd2.channel_names, scale=img_nd2.physical_pixel_sizes
)

# create features for napari label layer
#  (specifying the props_table will not recalculate the props,
#  i.e. show no intensity measurements)
features = visualise.create_napari_features(
    img_label=nuclei, img_intensity=img_nd2.get_data(),
    voxel_size=img_nd2.physical_pixel_sizes, props_table=property_table
)
# Adding both the filtered nuclei and the raw cp masks
visualise.add_labels(
    nuclei, name='nuclei', scale=img_nd2.physical_pixel_sizes,
    features=features
)

## Label mask modulations

In [ ]:
# Create cell an cytoplasm mask from nuclei
#  (expansion/shrinking in voxel_size units)
nuclei, cells, cyto = segment.create_cell_cyto_masks(
    nuclei, expansion=5, shrinking=2, voxel_size=img_nd2.physical_pixel_sizes
)

In [ ]:
show_napari_features = True
if show_napari_features:
    from time import time
    start = time()
    nuc_props = segment.measure_props(
        img_label=nuclei, img_intensity=img_nd2.get_data(),
        voxel_size=img_nd2.physical_pixel_sizes
    )
    cyto_props = segment.measure_props(
        img_label=cyto, img_intensity=img_nd2.get_data(),
        voxel_size=img_nd2.physical_pixel_sizes
    )
    cell_props = segment.measure_props(
        img_label=cells, img_intensity=img_nd2.get_data(),
        voxel_size=img_nd2.physical_pixel_sizes
    )
    nuc_feat = visualise.create_napari_features(props_table=nuc_props)
    cyto_feat = visualise.create_napari_features(props_table=cyto_props)
    cell_feat = visualise.create_napari_features(props_table=cell_props)
    print('Measuring compartment features took:', time() - start)
else:
    nuc_feat = None
    cyto_feat = None
    cell_feat = None

In [ ]:
visualise.add_labels(
    nuclei, name='nuc_shrunk', scale=img_nd2.physical_pixel_sizes,
    features=nuc_feat
)
visualise.add_labels(
    cyto, name='cyto', scale=img_nd2.physical_pixel_sizes,
    features=cyto_feat
)
visualise.add_labels(
    cells, name='cells', scale=img_nd2.physical_pixel_sizes,
    features=cell_feat
)

## Classification

In [ ]:
table = classify.create_base_table(
    img=img_nd2.get_data(), nuclei=nuclei, cyto=cyto, cells=cells,
    channel_names=img_nd2.channel_names, scale=img_nd2.physical_pixel_sizes
)


In [ ]:
table_mod = classify.classify(
    table, prop='Nucleus: projected_area', classification='big', min_val=74
)


In [ ]:
# Help function to list available classifications
classify.available_classifications(table_mod)

In [ ]:
table_mod2 = classify.classify(
    table_mod, prop='Nucleus: projected_perimeter', classification=None,
    min_val=25, show_counts=True
)

In [ ]:
# Show a specific class in napari (with a givein color)
big_nuclei = classify.show_by_class(
    table=table_mod2,
    class_names='big;Nucleus: projected_perimeter+',
    img_nuclei=nuclei,
    voxel_size=img_nd2.physical_pixel_sizes,
    color='blue'
    # gray, red, green, blue, magenta, cyan. Others will be random color
)

In [ ]:
# Estimate thresholds for different cell types
af488 = classify.estimate_threshold(
    'Cytoplasm: intensity_mean-AF488', table=table
)
af546 = classify.estimate_threshold(
    'Cytoplasm: intensity_mean-AF546', table=table
)
af647 = classify.estimate_threshold(
    'Cytoplasm: intensity_mean-AF647', table=table
)
print('Estimated thresholds:')
print('- AF488 =', af488)
print('- AF546 =', af546)
print('- AF647 =', af647)

In [ ]:
table_1 = classify.classify(
    table, prop='Cytoplasm: intensity_mean-AF488',
    classification='AF488+', min_val=200
)
table_1 = classify.classify(
    table_1, prop='Cytoplasm: intensity_mean-AF546',
    classification='AF546+', min_val=af546*0.9
)
table_1 = classify.classify(
    table_1, prop='Cytoplasm: intensity_mean-AF647',
    classification='AF647+', min_val=280, max_val=725, show_counts=True
)

In [ ]:
tripple_pos = classify.show_by_class(
    table_1, 'AF488+;AF546+', nuclei, 'magenta',
    img_nd2.physical_pixel_sizes
)

# Testing segmentation of scaffold

Li mehtod seems not too bad. maybe I can take the average Li threshold across the slices (of sum projected channel axis)

In [ ]:
path_full_nd2 = 'G:/20250211_MarcDu/24 C3 10X Large Image Z.nd2'

In [ ]:
img = io_utils.read_image(path_full_nd2)

In [ ]:
visualise.add_multichannel_image(
    img.get_dask_data(), name='scaffold',
    channel_names=img.channel_names, scale=img.physical_pixel_sizes
)

In [ ]:
img_small, binary, new_voxels = segex.scaffold_segmentation(
    img.get_dask_data(), voxel_size=img.physical_pixel_sizes, downscale=5
)

In [ ]:
visualise.add_image(img_small, name='downscaled', scale=new_voxels)
visualise.add_labels(binary, name='binary', scale=new_voxels)

In [ ]:
scaffold_area, scaffold_centroid = segex.get_area_and_center(
    binary, voxel_size=new_voxels
)

In [ ]:
visualise.add_point(
    scaffold_centroid, name='scaffold centroid', size=100
)